In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
def contraction(params,x,p):
    #beta and x are kind of parameters. x is the empirical distribution of x?
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:,0:k], params[0:k])  + params[k]*p[0]
    util2 = np.dot(x[:,k:], params[k+1:2*k+1]) + params[2*k+1]*p[1]
    contr_result = [np.exp(util1)/(1+np.exp(util1)),np.exp(util2)/(1+np.exp(util2))]
    return np.array(contr_result)


#actually caclualte an equilibrium in this game
N=1000
ps = np.array([.5,.5])
betas = np.array([1, 1,-2, 2, 1,-2])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
xs.insert(0,'x01',np.ones(N))
xs.insert(2,'x02',np.ones(N))
xs = np.array(xs)

print(contraction(betas,xs,ps))

[[0.31607933 0.37296382 0.60568578 ... 0.10661573 0.3171348  0.29971284]
 [0.93564081 0.42138761 0.99550664 ... 0.86939261 0.90211897 0.52030145]]


In [3]:
def contraction_map(betas,x,p):
    """final result is beliefs of firm1/firm2"""
    for i in range(20):
        #print('b1: %.4f, p2: %.4f'%tuple(p))
        p = contraction(betas,x,p).mean(axis=1)
        p = np.flip(p)
        #print('p1: %.4f, p2: %.4f'%tuple(p))
        #print('----- end of iteration %s ----'%i)
    return p

result_ps = contraction_map(betas,xs,ps)
us = np.random.logistic(size=(1000,2))

k = int(xs.shape[1]/2)
y1 = 1*(np.dot(xs[:,0:k], betas[0:k])  + betas[k]*result_ps[0] >= 0)
y2 = 1*(np.dot(xs[:,k:], betas[k+1:2*k+1]) + betas[2*k+1]*result_ps[1] >= 0)
ys = np.array([y1,y2]).transpose()
print(np.flip(result_ps))
print(ys.mean(axis=0))

[0.43317516 0.68736337]
[0.404 0.75 ]


In [4]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x01','x11','x02','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2  x01       x11  x02       x12
0    0.0  1.0  1.0 -0.771849  1.0  1.676752
1    0.0  0.0  1.0 -0.519523  1.0 -1.317080
2    1.0  1.0  1.0  0.429213  1.0  4.400650
3    1.0  0.0  1.0  2.886951  1.0 -5.231638
4    0.0  1.0  1.0 -1.351281  1.0  1.668818
..   ...  ...  ...       ...  ...       ...
995  1.0  1.0  1.0  0.881491  1.0 -0.923975
996  1.0  1.0  1.0  0.413658  1.0 -0.160997
997  0.0  1.0  1.0 -2.125786  1.0  0.895599
998  0.0  1.0  1.0 -0.766971  1.0  1.220994
999  0.0  1.0  1.0 -0.848666  1.0 -0.918750

[1000 rows x 6 columns]


In [15]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog.mean(axis=0)
        p = contraction_map(params,self.exog,p)
        
        likelihood = contraction(params,self.exog,p).transpose()
        ll = self.endog*np.log(likelihood) + (1-self.endog)*np.log(1-likelihood)
        
        return -1*ll.sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+3)
        start_params = np.array([1, 1,-2, 2, 1,-2])
        return super(BayesNashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x01','x11','x02','x12']]


model = BayesNashLogit(ys,xs)
model = model.fit()
print(model.summary(xname=["b01","b11","d1","b02","b12","d2"]))

/home/erichschulman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
/home/erichschulman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


Optimization terminated successfully.
         Current function value: 0.101044
         Iterations: 160
         Function evaluations: 517
                            BayesNashLogit Results                            
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -101.04
Model:                 BayesNashLogit   AIC:                             208.1
Method:            Maximum Likelihood   BIC:                             222.8
Date:                Mon, 08 Mar 2021                                         
Time:                        17:46:49                                         
No. Observations:                1000                                         
Df Residuals:                     997                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
